# Chapter 11 Lab

### Question 1: Select your products

Load the product spreadsheet from Chapter 10. Select 10 products from at least 3 different brands. Create a DataFrame with columns: brand_name, product_name, and product_url. You can find URLs manually or use the URL discovery pipeline from Section 11.4.

In [ ]:
import pandas as pdproducts = [    {"brand_name": "GORUCK", "product_name": "GR1 26L", "product_url": "https://www.goruck.com/products/gr1"},    {"brand_name": "GORUCK", "product_name": "Rucker 4.0 20L", "product_url": "https://www.goruck.com/products/rucker"},    {"brand_name": "GORUCK", "product_name": "Bullet Ruck 15L", "product_url": "https://www.goruck.com/products/bullet-ruck-15l"},    {"brand_name": "Osprey", "product_name": "Atmos AG 65", "product_url": "https://www.osprey.com/us/en/product/atmos-ag-65-ATMOS65S23.html"},    {"brand_name": "Osprey", "product_name": "Exos 58", "product_url": "https://www.osprey.com/us/en/product/exos-58-EXOS58F23.html"},    {"brand_name": "Petzl", "product_name": "Actik Core", "product_url": "https://www.petzl.com/US/en/Sport/PERFORMANCE-headlamps/ACTIK-CORE"},    {"brand_name": "Petzl", "product_name": "Tikka Core", "product_url": "https://www.petzl.com/US/en/Sport/TIKKA-CORE"},    {"brand_name": "MSR", "product_name": "Hubba Hubba NX 2", "product_url": "https://www.msrgear.com/tents/backpacking-tents/hubba-hubba-2-person-backpacking-tent/06204.html"},    {"brand_name": "Big Agnes", "product_name": "Copper Spur HV UL2", "product_url": "https://www.bigagnes.com/products/copper-spur-hv-ul2"},    {"brand_name": "Sawyer", "product_name": "Squeeze", "product_url": "https://www.sawyer.com/products/squeeze-water-filtration-system"},]df = pd.DataFrame(products)print(f"Selected {len(df)} products from {df['brand_name'].nunique()} brands")df

### Question 2: Fetch and clean HTML

For each product, fetch the HTML using requests and clean it using the aggressive cleaning approach from Section 11.5. Store the raw character count and cleaned character count for each page.

In [ ]:
import timefrom bs4 import BeautifulSoup# Use functions from guidehtml_data = []for _, row in df.iterrows():    try:        raw_html = requests.get(row["product_url"]).text        cleaned = clean_html_aggressive(raw_html)        html_data.append({            "product_name": row["product_name"],            "raw_chars": len(raw_html),            "cleaned_chars": len(cleaned),            "cleaned_text": cleaned,            "status": "fetched",        })    except Exception as e:        html_data.append({            "product_name": row["product_name"],            "raw_chars": 0,            "cleaned_chars": 0,            "cleaned_text": "",            "status": f"error: {type(e).__name__}",        })    time.sleep(1)html_df = pd.DataFrame(html_data)print(html_df[["product_name", "raw_chars", "cleaned_chars", "status"]])

### Question 3: Run AI extraction

Apply extract_product_with_ai to each cleaned page. Store the results in a DataFrame alongside the original product information and a status column tracking success or failure.

In [ ]:
results = []for i, row in df.iterrows():    record = row.to_dict()    record["status"] = "error"        html_row = html_data[i]    if html_row["status"] != "fetched":        record["status"] = html_row["status"]        results.append(record)        continue        try:        extraction = extract_product_with_ai(html_row["cleaned_text"])        record["extracted_name"] = extraction.product_name        record["extracted_brand"] = extraction.brand_name        record["extracted_price"] = extraction.price        record["extracted_weight"] = extraction.weight        record["extracted_category"] = extraction.category        record["extracted_description"] = (            extraction.description[:80] if extraction.description else None        )        record["status"] = "success"    except Exception as e:        record["status"] = f"error: {type(e).__name__}"        results.append(record)    time.sleep(1)results_df = pd.DataFrame(results)print(results_df[["product_name", "status", "extracted_price", "extracted_weight", "extracted_category"]])

### Question 4: Evaluate results

Build a summary DataFrame showing:
• How many products were successfully extracted
• For successful extractions, which fields were populated and which were null
• The overall "field coverage" (e.g., "price was extracted for 9 of 10 products")

In [ ]:
success_df = results_df[results_df["status"] == "success"]total = len(results_df)success_count = len(success_df)print(f"Total products:     {total}")print(f"Successful:         {success_count}")print(f"Failed:             {total - success_count}")print(f"Success rate:       {success_count / total:.0%}")# Field coveragefields = ["extracted_name", "extracted_brand", "extracted_price", "extracted_weight", "extracted_category", "extracted_description"]print("\nField coverage (of successful extractions):")for field in fields:    populated = success_df[field].notna().sum()    print(f"  {field}: {populated}/{success_count}")

### Question 5: Estimate costs

Using the cost estimation function from Section 11.8, calculate:
• Total input tokens across all 10 products
• Estimated total cost for the batch
• Projected cost if you scaled to the full 450-product spreadsheet

In [ ]:
total_input_tokens = 0for row in html_data:    if row["status"] == "fetched":        cost_info = estimate_extraction_cost(row["cleaned_text"])        total_input_tokens += cost_info["input_tokens"]avg_tokens = total_input_tokens / max(1, sum(1 for r in html_data if r["status"] == "fetched"))avg_cost = avg_tokens * (2.50 / 1_000_000) + 300 * (10.00 / 1_000_000)print(f"Total input tokens (10 products): {total_input_tokens:,}")print(f"Average tokens per product:       {avg_tokens:,.0f}")print(f"Estimated cost (10 products):     ${avg_cost * 10:.4f}")print(f"Projected cost (450 products):    ${avg_cost * 450:.2f}")

### Question 6: Compare to manual extraction

For at least 2 of your products, write manual extraction code (as in Chapter 10) and compare the results to the AI extraction. Which approach got more fields? Which was more accurate? Which took more time to write?

In [ ]:
# Manual extraction for GORUCK productsdef extract_manual_goruck_lab(text: str) -> dict:    """Manual extraction using simple patterns."""    import re        result = {        "product_name": None,        "brand_name": "GORUCK",  # Hardcoded        "price": None,        "weight": None,        "category": None,    }        # Extract price    price_match = re.search(r'\$[0-9,]+\.\d{2}', text)    if price_match:        result["price"] = price_match.group(0)        # Extract weight    weight_match = re.search(r'([0-9.]+\s*(lbs|oz))', text, re.I)    if weight_match:        result["weight"] = weight_match.group(1)        # Name is hardcoded    result["product_name"] = "GR1 26L"        return result# Compare for GR1gr1_cleaned = [h for h in html_data if h["product_name"] == "GR1 26L"][0]["cleaned_text"]manual_gr1 = extract_manual_goruck_lab(gr1_cleaned)ai_gr1_row = [r for r in results if r["product_name"] == "GR1 26L"][0]comparison = pd.DataFrame({    "Field": ["product_name", "brand_name", "price", "weight", "category"],    "Manual": [manual_gr1["product_name"], manual_gr1["brand_name"],               manual_gr1["price"], manual_gr1["weight"], manual_gr1["category"]],    "AI": [ai_gr1_row.get("extracted_name"), ai_gr1_row.get("extracted_brand"),           ai_gr1_row.get("extracted_price"), ai_gr1_row.get("extracted_weight"),           ai_gr1_row.get("extracted_category")],})print("GR1 26L Comparison:")print(comparison.to_string(index=False))print("\n✓ AI extracted more fields (category)")print("✓ Both got price and weight correctly")print("✓ Manual approach required site-specific code")print("✓ AI approach works across all brands without changes")